In [14]:
pip install pandas scikit-learn joblib

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from datetime import timedelta
import joblib

alunos_df = pd.read_csv("df_alunos_novo.csv", sep=",")  
checkins_df = pd.read_csv("df_checkins2.csv", sep=";")  

# Converte a coluna de datas para datetime
checkins_df["data"] = pd.to_datetime(checkins_df["data"], format="%d/%m/%Y")

# Encontra a data mais recente no dataset de checkins (necessário para calcular quanto tempo desde o ultimo checkin pois dataset usado para treino é antigo)
data_mais_recente = checkins_df["data"].max()
print("Data mais recente no dataset de check-ins:", data_mais_recente)

alunos_train, alunos_test = train_test_split(alunos_df, test_size=0.2, random_state=42)

# Função para calcular as features
def calcular_features_from_csv(alunos_df, checkins_df, data_referencia):
    """Calcula as features para o modelo de churn usando dados dos CSVs."""
    data = []

    checkins_dict = {}
    for _, checkin in checkins_df.iterrows():
        if checkin["aluno_id"] not in checkins_dict:
            checkins_dict[checkin["aluno_id"]] = []
        checkins_dict[checkin["aluno_id"]].append(checkin)

    for _, aluno in alunos_df.iterrows():
        # Checkins do aluno
        checkins_aluno = checkins_dict.get(aluno["id"], [])

        # Frequência semanal
        frequencia_semanal = sum(
            1 for checkin in checkins_aluno
            if checkin["data"] >= data_referencia - timedelta(days=7)
        )

        # Tempo desde o último checkin
        ultimo_checkin = max(
            (checkin["data"] for checkin in checkins_aluno),
            default=None
        )
        tempo_ultimo_checkin = (data_referencia - ultimo_checkin).days if ultimo_checkin else 30

        # Duração média das visitas
        duracoes = [
            (pd.to_datetime(checkin["horario_checkout"]).hour * 60 + pd.to_datetime(checkin["horario_checkout"]).minute) -
            (pd.to_datetime(checkin["horario_checkin"]).hour * 60 + pd.to_datetime(checkin["horario_checkin"]).minute)
            for checkin in checkins_aluno
            if not pd.isnull(checkin["horario_checkin"]) and not pd.isnull(checkin["horario_checkout"])
        ]
        duracao_media_visitas = sum(duracoes) / len(duracoes) if duracoes else 0

        # Tipo de plano
        PLANO_MAP = {"Basic": 1, "Student": 2, "Pro": 3}
        tipo_plano = PLANO_MAP.get(aluno["plano"], -1)  # Retorna -1 se o plano não estiver no mapeamento

        data.append([frequencia_semanal, tempo_ultimo_checkin, duracao_media_visitas, tipo_plano])

    return data

features_train = calcular_features_from_csv(alunos_train, checkins_df, data_mais_recente)
features_test = calcular_features_from_csv(alunos_test, checkins_df, data_mais_recente)

labels_train = alunos_train["risco_churn"].tolist()
labels_test = alunos_test["risco_churn"].tolist()

df_train = pd.DataFrame(features_train, columns=['frequencia_semanal', 'tempo_ultimo_checkin', 'duracao_media_visitas', 'tipo_plano'])
df_test = pd.DataFrame(features_test, columns=['frequencia_semanal', 'tempo_ultimo_checkin', 'duracao_media_visitas', 'tipo_plano'])

X_train = df_train
y_train = labels_train
X_test = df_test
y_test = labels_test

model = RandomForestClassifier(
    n_estimators=100,       
    max_depth=10,           
    min_samples_leaf=5,    
    random_state=42
)
model.fit(X_train, y_train)

# Avalie o modelo
y_pred = model.predict(X_test)
print("Acurácia no conjunto de teste:", accuracy_score(y_test, y_pred))
print("Relatório de classificação:\n", classification_report(y_test, y_pred))

# Salve o modelo treinado
joblib.dump(model, "churn_model.pkl")
print("Modelo de churn salvo como churn_model.pkl")

Data mais recente no dataset de check-ins: 2023-10-15 00:00:00
Acurácia no conjunto de teste: 0.712
Relatório de classificação:
               precision    recall  f1-score   support

           0       0.72      0.86      0.78       605
           1       0.69      0.49      0.57       395

    accuracy                           0.71      1000
   macro avg       0.71      0.67      0.68      1000
weighted avg       0.71      0.71      0.70      1000

Modelo de churn salvo como churn_model.pkl
